# MATH7013 Project
## [*Data*](https://github.com/FutureSharks/financial-data) provided by FutureSharks
## Program for preprocessing a dataset
### Implemented by Joel Thomas

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import pyfinancialdata as pyfd

### Load in the original dataset

In [2]:
# SPXUSD = S&P 500 futures priced in USD
df = pyfd.get(provider="histdata", instrument="SPXUSD", year=2017)["price"]
df.head()

date
2017-01-02 18:00:00    2243.50
2017-01-02 18:01:00    2243.00
2017-01-02 18:02:00    2243.25
2017-01-02 18:03:00    2243.00
2017-01-02 18:04:00    2243.75
Name: price, dtype: float64

### Preprocess the dataset - fill in missing values

In [3]:
i = 0
progress = 0
while i < len(df) - 1:
    curr_date = df.index[i]
    next_date = df.index[i+1]
    
    # Only forward fill missing data for gaps of less than 90 minutes (i.e. ignores weekends and public holidays)
    if (next_date - curr_date).total_seconds()/60 <= 90:
        # Check whether next minute's data already exists
        new_date = curr_date + dt.timedelta(minutes=1)
        if next_date != new_date:
            # Ignore daily trading halt periods (16:15 - 16:30 pm and 17:00 - 18:00 pm)
            if curr_date.time() != dt.time(hour=16, minute=15) and curr_date.time() != dt.time(hour=17, minute=0):
                # Use NaN for now
                new_row = {"price": np.nan}
                new_row = pd.Series(new_row, index=[new_date])
                new_row.index.names = ["date"]
                # Concatenate the new row into the existing dataset
                df = pd.concat([df.iloc[:i+1], new_row, df.iloc[i+1:]])
    
    # Track progress
    if i/(len(df) - 1) >= progress:
        print(f"Progress: {int(progress * 100)}%")
        progress += 0.05
    
    i += 1
    
print(f"Progress: {int(progress * 100)}%")

Progress: 0%
Progress: 5%
Progress: 10%
Progress: 15%
Progress: 20%
Progress: 25%
Progress: 30%
Progress: 35%
Progress: 40%
Progress: 44%
Progress: 49%
Progress: 54%
Progress: 60%
Progress: 65%
Progress: 70%
Progress: 75%
Progress: 80%
Progress: 85%
Progress: 90%
Progress: 95%
Progress: 100%


In [4]:
# Forward fill all NaNs
df.ffill(inplace=True)

### Generate remaining features - raw price changes of the last 45 min and momentum change to the previous 3 hours, 5 hours, 1 day, 3 days, and 10 days

In [5]:
# Convert series object to dataframe and rename column
df = df.to_frame(name="p_t")

df["z_t"] = df["p_t"].diff(periods=1)
for i in range(1, 45):
    col_name = f"z_(t-{i})"
    df[col_name] = df["z_t"].shift(periods=i)
    
for i in [3, 5]:
    lag = i*60
    col_name = f"m_(t-{lag})"
    df[col_name] = df["p_t"].diff(periods=lag)
    
for i in [1, 3, 10]:
    lag = i*24*60
    col_name = f"m_(t-{lag})"
    df[col_name] = df["p_t"].diff(periods=lag)

In [6]:
# Drop all rows that contain NaNs
df.dropna(inplace=True)
df.head()

p_t   z_t  z_(t-1)  z_(t-2)  z_(t-3)  z_(t-4)  \
date                                                                     
2017-01-17 10:00:00  2261.25  0.00    -0.75     0.00     0.25     0.25   
2017-01-17 10:01:00  2260.25 -1.00     0.00    -0.75     0.00     0.25   
2017-01-17 10:02:00  2260.75  0.50    -1.00     0.00    -0.75     0.00   
2017-01-17 10:03:00  2261.25  0.50     0.50    -1.00     0.00    -0.75   
2017-01-17 10:04:00  2261.50  0.25     0.50     0.50    -1.00     0.00   

                     z_(t-5)  z_(t-6)  z_(t-7)  z_(t-8)  ...  z_(t-40)  \
date                                                     ...             
2017-01-17 10:00:00     0.00     0.25     1.00    -0.25  ...      0.50   
2017-01-17 10:01:00     0.25     0.00     0.25     1.00  ...     -0.50   
2017-01-17 10:02:00     0.25     0.25     0.00     0.25  ...      0.00   
2017-01-17 10:03:00     0.00     0.25     0.25     0.00  ...      0.00   
2017-01-17 10:04:00    -0.75     0.00     0.25     0.25  ...     -0.75   

                     z_(t-41)  z_(t-42)  z_(t-43)  z_(t-44)  m_(t-180)  \
date                                                                     
2017-01-17 10:00:00       0.0       0.0      0.25     -0.75       0.25   
2017-01-17 10:01:00       0.5       0.0      0.00      0.25      -0.50   
2017-01-17 10:02:00      -0.5       0.5      0.00      0.00      -0.50   
2017-01-17 10:03:00       0.0      -0.5      0.50      0.00      -0.50   
2017-01-17 10:04:00       0.0       0.0     -0.50      0.50      -1.75   

                     m_(t-300)  m_(t-1440)  m_(t-4320)  m_(t-14400)  
date                                                                 
2017-01-17 10:00:00      -1.25       -6.00       -2.75        17.75  
2017-01-17 10:01:00      -2.25       -7.00       -3.75        17.25  
2017-01-17 10:02:00      -2.00       -6.25       -3.25        17.50  
2017-01-17 10:03:00      -1.50       -6.00       -2.75        18.25  
2017-01-17 10:04:00      -1.50       -5.75       -2.50        17.75  

[5 rows x 51 columns]

### Save the final dataset 

In [7]:
df.to_csv("SPXUSD_2017.csv")